In [45]:
import os
from dotenv import load_dotenv
from pathlib import Path
from openai import OpenAI
import requests

# Get current working directory (works in notebooks)
cwd = Path().resolve()

env_path = cwd / ".env"
if not env_path.exists():
    env_path = cwd.parent / ".env"

load_dotenv(dotenv_path=env_path)
openai_api = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api)


NEWS_API_KEY = os.getenv("NEWS_API_KEY")


# Check it loaded correctly
print("✅ NewsAPI key loaded:", bool(NEWS_API_KEY))


✅ NewsAPI key loaded: True


In [46]:
def fetch_africa_news():
    url = f"https://newsapi.org/v2/everything?q=africa+startup+funding&language=en&sortBy=publishedAt&pageSize=50&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    data = response.json()
    print(" Response JSON:", data.get("status", "unknown"))
    return data.get("articles", [])


In [47]:
news = fetch_africa_news()
print(f"Fetched {len(news)} articles")


 Response JSON: ok
Fetched 50 articles


In [48]:
def filter_africa_stories(articles, limit=10):
    keywords = ["africa", "nigeria", "kenya", "ghana", "south africa", "startup", "funding", "fintech", "investment"]
    filtered = []
    for article in articles:
        title = (article.get("title") or "").lower()
        description = (article.get("description") or "").lower()
        combined = f"{title} {description}"
        if any(k in combined for k in keywords):
            filtered.append(article)
    print(f"Filtered {len(filtered)} stories")  # debug line
    return filtered[:limit]


In [50]:
def summarize_article(title, content):
    prompt = f"Summarise this African startup news article in 200 words:\n\nTitle: {title}\n\nContent: {content}"
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content.strip()

In [51]:
if __name__ == "__main__":
    print("Fetching latest Africa startup news...")
    news = fetch_africa_news()
    print(f"Fetched {len(news)} articles")

    filtered_news = filter_africa_stories(news)
    print(f"Filtered to {len(filtered_news)} Africa-related stories")

    summaries = []
    for item in filtered_news[:10]:
        summary = summarize_article(item["title"], item.get("description", ""))
        summaries.append({
            "title": item["title"],
            "url": item["url"],
            "summary": summary
        })

    print("\n📰 Top Summaries:")
    for i, s in enumerate(summaries, 1):
        print(f"\n{i}. {s['title']}\n{s['summary']}\n{s['url']}")

Fetching latest Africa startup news...
 Response JSON: ok
Fetched 50 articles
Filtered 20 stories
Filtered to 10 Africa-related stories

📰 Top Summaries:

1. New “Salt Battery” Proves Energy Storage Exists, Again
Aslym Energy, a US startup, is set to launch a pioneering sodium-ion energy storage solution designed to enhance and expedite the global transition to renewable energy sources. This innovative "salt battery" technology addresses the pressing need for efficient energy storage systems that can store renewable energy generated from sources like solar and wind. By leveraging sodium, a more abundant and environmentally friendly alternative to lithium, Aslym Energy aims to provide a sustainable and cost-effective solution to the challenges faced in energy storage. As the world moves toward cleaner energy, effective storage solutions are crucial for managing supply and demand, especially as reliance on renewable sources increases. The introduction of sodium-ion batteries may help ove

In [1]:
import sys
from pathlib import Path

# Find the project root (one level above this notebook)
root_dir = Path().resolve().parent  # if notebook is inside /notebooks
print("Root directory:", root_dir)

# Add the root to sys.path if not already there
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Verify
print("sys.path now contains:", sys.path[-3:])


Root directory: C:\Users\hamza\OneDrive\Documents\afripulse-ai
sys.path now contains: ['C:\\Users\\hamza\\OneDrive\\Documents\\afripulse-ai\\venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\hamza\\OneDrive\\Documents\\afripulse-ai\\venv\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\hamza\\OneDrive\\Documents\\afripulse-ai']


In [2]:
from dotenv import load_dotenv
from pathlib import Path
import os
from openai import OpenAI
import requests

# Point to your project root
env_path = Path().resolve().parent / ".env"  # if notebook is inside /notebooks
if not env_path.exists():
    env_path = Path().resolve() / ".env"  # if notebook is already in root

load_dotenv(dotenv_path=env_path)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

NEWS_API_KEY = os.getenv("NEWS_API_KEY")
print("✅ OPENAI_API_KEY loaded:", bool(os.getenv("OPENAI_API_KEY")))


✅ OPENAI_API_KEY loaded: True


In [3]:
from src.fetchNews import fetch_africa_news
articles = fetch_africa_news()


Response status: ok
Successfully fetched 50 articles.
Saved a local copy of fetched news.


In [9]:
from src.fetchNews import fetch_africa_news
from src.filterNews import filter_africa_stories
from src.sumarizeNews import summarize_article

articles = fetch_africa_news()
filtered = filter_africa_stories(articles)
print(len(filtered))

summary = summarize_article(filtered[0]["title"], filtered[0]["description"])
print(summary)


Response status: ok
Successfully fetched 50 articles.
Saved a local copy of fetched news.
Filtered 20 stories
10
US startup Aslym Energy is introducing a sodium-ion energy storage solution, referred to as a "salt battery," to enhance energy storage capabilities. This innovation is designed to support and accelerate the transition to renewable energy globally. The launch underscores the growing importance of effective energy storage technology in combating climate change.


In [10]:
import datetime
from pathlib import Path

def generate_weekly_digest(summaries):
    today = datetime.date.today().strftime("%B %d, %Y")
    md_content = f"# AfriPulse Weekly Digest – {today}\n\n"

    for i, story in enumerate(summaries, 1):
        md_content += f"### {i}. [{story['title']}]({story['url']})\n{story['summary']}\n\n"

    output_dir = Path(__file__).resolve().parent.parent / "outputs"
    output_dir.mkdir(exist_ok=True)
    output_path = output_dir / "weeklyDigest.md"

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(md_content)

    print(f"Weekly digest saved to {output_path}")


In [8]:
!streamlit run ../dashboard.py


^C
